# Pneumococcus modelling codes 

_Streptococcus pneumoniae_ (or 'pneumococcus') simulation codes used to explore possible transmission structures and dynamical properties.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as spec
import time

### Single strain per person and exponential recovery time model

Start with a simple Markovian (exponentially-distributed in event time) recovery rate for each strain. This first model will be based on the household-community split (as in [https://academic.oup.com/aje/article/166/2/228/98916]) with the following transmission probabilities for an individuals (who we shall assume can only have a single strain at a time) within household $h$ of age $a$, and the $i$-th strain, over time interval $\delta t$

$${\rm Pr}({\cal I}^h_{ai}\rightarrow {\cal I}^h_{ai}-1,S^h_{a}\rightarrow S^h_{a}+1,\delta t) = \mu_{ai}{\cal I}^h_{ai}(t)\delta t$$
$${\rm Pr}({\cal I}^h_{ai}\rightarrow {\cal I}^h_{ai}+1,S^h_{a}\rightarrow S^h_{a}-1,\delta t)  = \Lambda_{ai}(t)\delta t$$
$${\rm Pr}({\cal I}^h_{ai}\rightarrow {\cal I}^h_{ai}+1,{\cal I}^h_{ai'}\rightarrow {\cal I}^h_{ai'}-1,\delta t) = f_{i'i}\Lambda_{ai}(t)\delta t$$
$${\rm Pr}({\cal I}^h_{ai}\rightarrow {\cal I}^h_{ai}-1,{\cal I}^h_{ai'}\rightarrow {\cal I}^h_{ai'}+1,\delta t) = f_{ii'}\Lambda_{ai'}(t)\delta t$$

where ${\cal I}^h_{ai}(t)$ denotes the number of infective individuals of age $a$ with strain $i$ in the household, $S^h_{a}(t)$ denotes the number of susceptible individuals of age $a$ in the household, $f_{ii'}$ is the (asymmetric) strain competition matrix and $\mu_{ai}$ denotes the recovery rate of an individual of age $a$ from the $i$-th strain. The force of infection $\Lambda_{ai}(t)$ is split into household $\lambda^h_{ai}(t)$ (with $N_h$ individuals living within it) and community $\lambda^{\rm com}_{ai}(t)$ parts in the following way

$$\Lambda^h_{ai}(t)=\lambda^h_{ai}(t)+\lambda^{h-{\rm com}}_{ai}(t)= \frac{1}{N_h-1}\sum_{\forall a'} \beta^{a'}_{ai}{\cal I}^h_{a'i}(t) + \frac{\alpha}{(\sum_{\forall h}N_h)-N_h}\sum_{\forall a'}\sum_{\forall h' \neq h} \beta^{a'}_{ai}{\cal I}^{h'}_{a'i}(t)\,.$$

In the above expression, $\beta^{a'}_{ai}$ is the contact rate between individuals of ages $a$ and $a'$ for the $i$-th strain, and $\alpha$ (<1) encodes the damping of community contact relative to the household. For computation speed, we can redefine the community term in the force of infection above for an individual in terms of the of the 'total community force of infection' $L_{ai}(t)$ for each age group and strain like so

$$\lambda^{h-{\rm com}}_{ai}(t) = \frac{\alpha}{(\sum_{\forall h}N_h)-N_h}\left[ L_{ai}(t) -\sum_{\forall a'}\beta^{a'}_{ai}{\cal I}^{h}_{a'i}(t) \right]\,,$$

where $L_{ai}(t)$ is defined as

$$L_{ai}(t) \equiv \sum_{\forall a'}\sum_{\forall h} \beta^{a'}_{ai}{\cal I}^{h}_{a'i}(t)\,,$$

such that 

$$\Lambda^h_{ai}(t)=\left[ \frac{1}{N_h-1} -\frac{\alpha}{(\sum_{\forall h}N_h)-N_h}\right]\sum_{\forall a'} \beta^{a'}_{ai}{\cal I}^h_{a'i}(t) + \frac{\alpha}{(\sum_{\forall h}N_h)-N_h}L_{ai}(t)\,.$$

A simple code which corresponds to this system is below. The key parameters are set first here...

In [32]:
number_of_households = 100
number_of_strains = 100
demographic_household_fractions = [0.3,0.4,0.3] # [Children,Adults,Elderly]
alpha_damping = 0.1
number_of_realisations = 100
realisations_per_run = 10                       # Should be a factor of the number of realisations
time_at_beginning = 0.0                         # In units of years
time_at_end = 10.0                              # In units of years
do_nothing_timescale = 0.1                      # In units of years

...then the contact rates, recovery rates and competition matrix...

In [33]:
children_to_adult_contact_rates = np.ones(number_of_strains)
children_to_elderly_contact_rates = np.ones(number_of_strains)
adult_to_elderly_contact_rates = np.ones(number_of_strains)
competition_matrix = np.ones((number_of_strains,number_of_strains))
recovery_rates = np.ones(number_of_strains)

...then the initial prevalences for all strains...

In [34]:
initial_prevalences = np.random.uniform(size=number_of_strains)

...and finally the algorithm is...

In [9]:
# Draw the number of people within each household
household_people_numbers = np.random.randint(1,10,size=number_of_households)
total_number_of_people = np.sum(household_people_numbers)

# Set the demographic numbers within each household
demographic_numbers = np.asarray([np.random.multinomial(hp,demographic_household_fractions) \
                                  for hp in household_people_numbers]).T
number_of_children_in_each_household = demographic_numbers[0]
number_of_adults_in_each_household = demographic_numbers[1]
number_of_elderly_in_each_household = demographic_numbers[2]

# Generate initial matrices of infectives - dimension 0 is the household ID, 
# dimension 1 is the strain and dimension 2 is the realisation
household_children_infectives = np.random.binomial(np.tensordot(number_of_children_in_each_household,\
                                                   np.ones((number_of_strains,realisations_per_run)),axes=0),\
                                                   np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                   initial_prevalences,axes=0),np.ones(realisations_per_run),axes=0),\
                                                   size=(number_of_households,number_of_strains,realisations_per_run))
household_adult_infectives = np.random.binomial(np.tensordot(number_of_adults_in_each_household,\
                                                   np.ones((number_of_strains,realisations_per_run)),axes=0),\
                                                   np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                   initial_prevalences,axes=0),np.ones(realisations_per_run),axes=0),\
                                                   size=(number_of_households,number_of_strains,realisations_per_run))
household_elderly_infectives = np.random.binomial(np.tensordot(number_of_elderly_in_each_household,\
                                                   np.ones((number_of_strains,realisations_per_run)),axes=0),\
                                                   np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                   initial_prevalences,axes=0),np.ones(realisations_per_run),axes=0),\
                                                   size=(number_of_households,number_of_strains,realisations_per_run))

# Generate the corresponding matrices of susceptibles 
household_children_susceptibles = np.tensordot(number_of_children_in_each_household,\
                                  np.ones((number_of_strains,realisations_per_run)),axes=0) - \
                                  household_children_infectives
household_adult_susceptibles = np.tensordot(number_of_adults_in_each_household,\
                                  np.ones((number_of_strains,realisations_per_run)),axes=0) - \
                                  household_adult_infectives
household_elderly_susceptibles = np.tensordot(number_of_elderly_in_each_household,\
                                  np.ones((number_of_strains,realisations_per_run)),axes=0) - \
                                  household_elderly_infectives

# Compute the number of runs required and initialise main loop
number_of_runs = number_of_realisations/realisations_per_run
for n in range(0,len(number_of_runs)):

    # Initialise time and run over realisations
    t = time_at_beginning*np.ones(realisations_per_run)
    terminated_realisations_per_run = np.ones(realisations_per_run)
    while slowest_t < time_at_end:
    
        # Iterate forward in time 
        t += terminated_realisations_per_run*np.exponential(scale=do_nothing_timescale,size=realisations_per_run)
    
        # Set slowest time
        slowest_t = np.ndarray.min(t)
    
        # Event realisations at this timestep
        event_realisations = np.random.uniform(size=(number_of_households,number_of_strains,realisations_per_run))
        
        # Total community forces of infection for each age group
        children_total_community_force_of_infection=np.sum((np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                             children_to_adult_contact_rates,axes=0),\
                                                             np.ones(realisations_per_run),axes=0)*\
                                                             household_adult_infectives) + \
                                                            (np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                             children_to_elderly_contact_rates,axes=0),\
                                                             np.ones(realisations_per_run),axes=0)*\
                                                             household_elderly_infectives),axis=0)
        adult_total_community_force_of_infection=np.sum((np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                             children_to_adult_contact_rates,axes=0),\
                                                             np.ones(realisations_per_run),axes=0)*\
                                                             household_children_infectives) + \
                                                            (np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                             adult_to_elderly_contact_rates,axes=0),\
                                                             np.ones(realisations_per_run),axes=0)*\
                                                             household_elderly_infectives),axis=0)
        elderly_total_community_force_of_infection=np.sum((np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                             children_to_elderly_contact_rates,axes=0),\
                                                             np.ones(realisations_per_run),axes=0)*\
                                                             household_children_infectives) + \
                                                            (np.tensordot(np.tensordot(np.ones(number_of_households),\
                                                             adult_to_elderly_contact_rates,axes=0),\
                                                             np.ones(realisations_per_run),axes=0)*\
                                                             household_adult_infectives),axis=0)
        
        # Recovery event rates for each age group
        children_recovery_event_rates = np.tensordot(household_children_infectives,\
                                        np.tensordot(recovery_rates,np.ones(realisations_per_run),axes=0),axes=0)
        adult_recovery_event_rates = np.tensordot(household_adult_infectives,\
                                     np.tensordot(recovery_rates,np.ones(realisations_per_run),axes=0),axes=0)
        elderly_recovery_event_rates = np.tensordot(household_elderly_infectives,\
                                       np.tensordot(recovery_rates,np.ones(realisations_per_run),axes=0),axes=0)
        
        # New carriage event rates for each age group (also the forces of infection)
        children_new_carriage_event_rates=((np.tensordot((household_people_numbers>1)/((household_people_numbers>1)*\
                                            household_people_numbers.astype(float)-1.0),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0)-\
                                            np.tensordot(alpha_damping/(float(total_number_of_people)-\
                                            household_people_numbers.astype(float)),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0))*\
                                            ((np.tensordot(np.tensordot(np.ones(number_of_households),\
                                            children_to_adult_contact_rates,axes=0),\
                                            np.ones(realisations_per_run),axes=0)*\
                                            household_adult_infectives) + \
                                            (np.tensordot(np.tensordot(np.ones(number_of_households),\
                                            children_to_elderly_contact_rates,axes=0),\
                                            np.ones(realisations_per_run),axes=0)*\
                                            household_elderly_infectives))) + \
                                            (np.tensordot(alpha_damping/(float(total_number_of_people)-\
                                            household_people_numbers.astype(float)),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0)*\
                                            (np.tensordot(np.ones(number_of_households),\
                                            children_total_community_force_of_infection,axes=0)))   
        adult_new_carriage_event_rates=((np.tensordot((household_people_numbers>1)/((household_people_numbers>1)*\
                                            household_people_numbers.astype(float)-1.0),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0)-\
                                            np.tensordot(alpha_damping/(float(total_number_of_people)-\
                                            household_people_numbers.astype(float)),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0))*\
                                            ((np.tensordot(np.tensordot(np.ones(number_of_households),\
                                            children_to_adult_contact_rates,axes=0),\
                                            np.ones(realisations_per_run),axes=0)*\
                                            household_children_infectives) + \
                                            (np.tensordot(np.tensordot(np.ones(number_of_households),\
                                            adult_to_elderly_contact_rates,axes=0),\
                                            np.ones(realisations_per_run),axes=0)*\
                                            household_elderly_infectives))) + \
                                            (np.tensordot(alpha_damping/(float(total_number_of_people)-\
                                            household_people_numbers.astype(float)),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0)*\
                                            (np.tensordot(np.ones(number_of_households),\
                                            adult_total_community_force_of_infection,axes=0)))
        elderly_new_carriage_event_rates=((np.tensordot((household_people_numbers>1)/((household_people_numbers>1)*\
                                            household_people_numbers.astype(float)-1.0),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0)-\
                                            np.tensordot(alpha_damping/(float(total_number_of_people)-\
                                            household_people_numbers.astype(float)),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0))*\
                                            ((np.tensordot(np.tensordot(np.ones(number_of_households),\
                                            children_to_elderly_contact_rates,axes=0),\
                                            np.ones(realisations_per_run),axes=0)*\
                                            household_children_infectives) + \
                                            (np.tensordot(np.tensordot(np.ones(number_of_households),\
                                            adult_to_elderly_contact_rates,axes=0),\
                                            np.ones(realisations_per_run),axes=0)*\
                                            household_adult_infectives))) + \
                                            (np.tensordot(alpha_damping/(float(total_number_of_people)-\
                                            household_people_numbers.astype(float)),\
                                            np.ones((number_of_strains,realisations_per_run)),axes=0)*\
                                            (np.tensordot(np.ones(number_of_households),\
                                            elderly_total_community_force_of_infection,axes=0)))
        
        # Sum over possible events that can occur within each household
        sum_of_event_rates_in_each_household = (1.0/do_nothing_timescale) + children_recovery_event_rates + \
                                               adult_recovery_event_rates + elderly_recovery_event_rates + \
                                               children_new_carriage_event_rates + adult_new_carriage_event_rates + \
                                               elderly_new_carriage_event_rates +# Sum over all possible strain exchanges
        
        # Trigger termination for the runs which have ended
        terminated_realisations_per_run = (t < time_at_end)

1. One interesting possibility is to numerically find the metastable equilibria in the determinsitic model and compute the local variance that would exist at them all - this should give some notion of probability of being at a particular set of values in the full phase space.
2. Another interesting possibility is to use random matrix theory in solving this system given how many strains there are!